In [1]:
pip install faster-whisper

In [2]:
import logging

logging.basicConfig()
logging.getLogger("faster_whisper").setLevel(logging.DEBUG)

In [3]:
from faster_whisper import WhisperModel
model_size = "large-v3"
model = WhisperModel(model_size, device="cuda", compute_type="float16")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
from faster_whisper import WhisperModel

model_size = "large-v3"

# Run on GPU with FP16
model = WhisperModel(model_size, device="cuda", compute_type="float16")

# or run on GPU with INT8
# model = WhisperModel(model_size, device="cuda", compute_type="int8_float16")
# or run on CPU with INT8
# model = WhisperModel(model_size, device="cpu", compute_type="int8")

segments, info = model.transcribe("audio.mp3", beam_size=5)

print("Detected language '%s' with probability %f" % (info.language, info.language_probability))

for segment in segments:
    print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
INFO:faster_whisper:Processing audio with duration 02:03.717
INFO:faster_whisper:Detected language 'en' with probability 0.74
DEBUG:faster_whisper:Processing segment at 00:00.000


Detected language 'en' with probability 0.735840


DEBUG:faster_whisper:Processing segment at 00:26.440


[0.00s -> 5.20s]  Organize, every other day I organize
[5.20s -> 26.44s]  Organize, every other day I organize


DEBUG:faster_whisper:Processing segment at 00:56.440


[26.44s -> 30.74s]  Organize, some of us are wise
[30.74s -> 34.98s]  Every other day see who are wise
[34.98s -> 41.84s]  Oh Laquanju, oh pa, oh pa, oh pa
[41.84s -> 43.32s]  You are no banju
[43.32s -> 49.46s]  Oh Paquanju, oh pa, oh pa
[49.46s -> 51.82s]  Oh Laquanju, oh si Laquanju
[54.82s -> 56.42s]  There's a one and two for Laquanju
[56.42s -> 56.44s]  There's a one and two for Laquanju


DEBUG:faster_whisper:Compression ratio threshold is not met with temperature 0.0 (9.979592 > 2.400000)
DEBUG:faster_whisper:Processing segment at 01:26.420


[56.44s -> 86.42s]  There's a one and two for Laquanju


DEBUG:faster_whisper:Processing segment at 01:56.400


[86.42s -> 89.18s]  Anything I want to, I will do
[89.18s -> 94.02s]  Organize, every other day I organize
[94.02s -> 98.68s]  Organize, some of us are wise
[98.68s -> 102.90s]  Every other day see who are wise
[102.90s -> 109.80s]  Oh Laquanju, oh pa, oh pa, oh pa
[109.80s -> 111.26s]  You are no banju
[111.26s -> 116.40s]  Oh Paquanju, oh pa, oh pa
[116.40s -> 119.16s]  Oh Laquanju, oh pa, oh pa, oh pa
[119.16s -> 120.16s]  You are no banju
[120.16s -> 123.70s]  Oh Laquanju, oh pa, oh pa, oh pa


In [4]:
segments, info = model.transcribe("new.m4a", beam_size=5)
for segment in segments:
    print(segment.text)

INFO:faster_whisper:Processing audio with duration 00:06.920
INFO:faster_whisper:Detected language 'yo' with probability 0.49
DEBUG:faster_whisper:Processing segment at 00:00.000


 Ok, so Grace, this meeting is hereby commenced and we will begin in earnest.


In [5]:
pip install pyngrok

In [6]:
import os
import threading
from flask import Flask, request, send_file
from werkzeug.utils import secure_filename
from pyngrok import ngrok

app = Flask(__name__)
port = "5000"

# Open a ngrok tunnel to the HTTP server
ngrok.set_auth_token("1jRLTRPuF8Iaa8nwWxB4gFRFEJZ_2WFCFQ1wZNgBqPUmearFN")
public_url = ngrok.connect(port).public_url
print(f"{public_url} ->", f"http://127.0.0.1:{port}")

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# Define directory to save uploaded files
upload_folder = 'uploads'  # Create this folder if it doesn't exist

# Define Flask routes
@app.route("/")
def index():
    return """
    <h1>Upload Audio File</h1>
    <form method="post" enctype="multipart/form-data" action="upload_file">
      <input type="file" name="audio_file">
      <input type="submit">
    </form>
    """

@app.route("/upload_file", methods=['POST'])
def upload_file():
    if 'audio_file' not in request.files:
        return "No file selected"

    file = request.files['audio_file']
    filename = secure_filename(file.filename)
    filepath = os.path.join(upload_folder, filename)
    file.save(filepath)

    segments, info = model.transcribe(f"uploads/{filename}", beam_size=5)
    text = [segment.text for segment in segments]
    return text

    #return f"File uploaded successfully! <a href='{os.path.join(app.config['BASE_URL'], upload_folder, filename)}'>Download</a>"

# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()


https://b960-34-168-62-102.ngrok-free.app -> http://127.0.0.1:5000
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [36]:
import os
import threading
from flask import Flask, request, send_file
from werkzeug.utils import secure_filename
from pyngrok import ngrok

app = Flask(__name__)
port = "6056"

# Open a ngrok tunnel to the HTTP server
ngrok.set_auth_token("1jRLTRPuF8Iaa8nwWxB4gFRFEJZ_2WFCFQ1wZNgBqPUmearFN")
public_url = ngrok.connect(port).public_url
print(f"{public_url} ->", f"http://127.0.0.1:{port}")

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# Define directory to save uploaded files
upload_folder = 'uploads'  # Create this folder if it doesn't exist

# Define Flask routes
@app.route("/")
def index():
    return """
    <h1>Upload Audio File</h1>
    <form method="post" enctype="multipart/form-data">
      <input type="file" name="audio_file">
      <input type="submit">
    </form>
    """

@app.route("/", methods=['POST'])
def upload_file():
    if 'audio_file' not in request.files:
        return "No file selected"

    file = request.files['audio_file']
    # Validate file format (optional)
    if file.mimetype not in ['audio/wav', 'audio/mpeg']:
        return "Invalid file format. Please upload wav or mp3 files."

    filename = secure_filename(file.filename)
    filepath = os.path.join(upload_folder, filename)
    file.save(filepath)

    segments, info = model.transcribe(f"uploads/{filename}", beam_size=5)
    text = [segment.text for segment in segments]
    return text

    #return f"File uploaded successfully! <a href='{os.path.join(app.config['BASE_URL'], upload_folder, filename)}'>Download</a>"

# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()


https://72e0-34-168-62-102.ngrok-free.app -> http://127.0.0.1:6056
 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


In [32]:
!ps aux | grep "flask"


root        8486  0.0  0.0   7376  3472 ?        S    10:37   0:00 /bin/bash -c ps aux | grep "flask
root        8488  0.0  0.0   6484  2288 ?        S    10:37   0:00 grep flask


In [29]:
!kill 3576

/bin/bash: line 1: kill: (3576) - No such process


In [8]:
!fuser -n tcp 5000



5000/tcp:            10675


In [1]:
import os
import threading
from flask import Flask, request, send_file
from werkzeug.utils import secure_filename
from pyngrok import ngrok

app = Flask(__name__)
port = "6060"  # Change port to 6060

# Open a ngrok tunnel to the HTTP server
ngrok.set_auth_token("1jRLTRPuF8Iaa8nwWxB4gFRFEJZ_2WFCFQ1wZNgBqPUmearFN")
public_url = ngrok.connect(port).public_url
print(f"{public_url} ->", f"http://127.0.0.1:{port}")

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# Define directory to save uploaded files
upload_folder = 'uploads'  # Create this folder if it doesn't exist

# Define Flask routes
@app.route("/")
def index():
    return """
    <h1>Upload Audio File</h1>
    <form method="post" enctype="multipart/form-data">
      <input type="file" name="audio_file">
      <input type="submit">
    </form>
    """

@app.route("/", methods=['POST'])
def upload_file():
    if 'audio_file' not in request.files:
        return "No file selected"

    file = request.files['audio_file']
    # Validate file format (optional)
    if file.mimetype not in ['audio/wav', 'audio/mpeg']:
        return "Invalid file format. Please upload wav or mp3 files."

    filename = secure_filename(file.filename)
    filepath = os.path.join(upload_folder, filename)
    file.save(filepath)

    # Replace this section with your audio transcription logic (model.transcribe)
    segments, info = model.transcribe(f"uploads/{filename}", beam_size=5)
    text = [segment.text for segment in segments]
    return text

    #return f"File uploaded successfully! <a href='{os.path.join(app.config['BASE_URL'], upload_folder, filename)}'>Download</a>"

# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()


https://19a1-34-168-62-102.ngrok-free.app -> http://127.0.0.1:6060
 * Serving Flask app '__main__'
 * Debug mode: off


In [9]:
pip install fastapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [13]:
import os
from fastapi import FastAPI, File, UploadFile
from werkzeug.utils import secure_filename
from pyngrok import ngrok

app = FastAPI()
port= "6060"

# Open a ngrok tunnel to the HTTP server, specifying port 6060
ngrok.set_auth_token("1jRLTRPuF8Iaa8nwWxB4gFRFEJZ_2WFCFQ1wZNgBqPUmearFN")
public_url = ngrok.connect(port).public_url
print(f"{public_url} ->", f"http://127.0.0.1:6060")

# Define directory to save uploaded files
upload_folder = 'uploads'  # Create this folder if it doesn't exist


@app.post("/")
async def upload_file(audio_file: UploadFile):
    if not audio_file.filename:
        return "No file selected"

    # Validate file format (optional)
    if audio_file.content_type not in ['audio/wav', 'audio/mpeg']:
        return "Invalid file format. Please upload wav or mp3 files."

    filename = secure_filename(audio_file.filename)
    filepath = os.path.join(upload_folder, filename)
    with open(filepath, 'wb') as f:
        await f.write(audio_file.file)

    # Replace this section with your audio transcription logic (model.transcribe)
    segments, info = model.transcribe(filepath, beam_size=5)
    text = [segment.text for segment in segments]
    return text

if __name__ == "__main__":
    import uvicorn
    uvicorn.run("app", host="0.0.0.0", port=6060)


https://efc6-34-168-62-102.ngrok-free.app -> http://127.0.0.1:6060


RuntimeError: asyncio.run() cannot be called from a running event loop

In [14]:
pip install cherrypy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.8/348.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 6.0 MB/s eta 0:00:00


In [23]:
import os
import threading
from werkzeug.utils import secure_filename
from pyngrok import ngrok
import cherrypy

config = {'/':
    {
        'server.socket_host': '0.0.0.0',
        'server.socket_port': 6060,
    }
}

class UploadServer(object):
    @cherrypy.expose
    def index(self):
        return """
        <h1>Upload Audio File</h1>
        <form method="post" enctype="multipart/form-data" action="upload_file">
          <input type="file" name="audio_file">
          <input type="submit">
        </form>
        """

    @cherrypy.expose
    def upload_file(self, audio_file):
        if not audio_file.filename:
            return "No file selected"

        file = request.files['audio_file']
        filepath = os.path.join('uploads', file.filename)
        file.save()

        # Replace this section with your audio transcription logic (model.transcribe)
        segments, info = model.transcribe(f"uploads/{file.filename}", beam_size=5)
        text = [segment.text for segment in segments]
        return text

if __name__ == "__main__":
    # Open a ngrok tunnel to the HTTP server
    ngrok.set_auth_token("1jRLTRPuF8Iaa8nwWxB4gFRFEJZ_2WFCFQ1wZNgBqPUmearFN")
    public_url = ngrok.connect(6060).public_url
    print(f"{public_url} ->", f"http://127.0.0.1:6060")

    # Define directory to save uploaded files
    upload_folder = 'uploads'  # Create this folder if it doesn't exist

    # Start CherryPy server in a new thread
    cherrypy.config.update({'server.socket_host': '0.0.0.0', 'server.socket_port': 6060})
    cherrypy.quickstart(UploadServer(), '/', config=config)


[12/Jun/2024:11:18:13] ENGINE Listening for SIGTERM.
INFO:cherrypy.error:[12/Jun/2024:11:18:13] ENGINE Listening for SIGTERM.
[12/Jun/2024:11:18:13] ENGINE Listening for SIGHUP.
INFO:cherrypy.error:[12/Jun/2024:11:18:13] ENGINE Listening for SIGHUP.
[12/Jun/2024:11:18:13] ENGINE Listening for SIGUSR1.
INFO:cherrypy.error:[12/Jun/2024:11:18:13] ENGINE Listening for SIGUSR1.
[12/Jun/2024:11:18:13] ENGINE Bus STARTING
INFO:cherrypy.error:[12/Jun/2024:11:18:13] ENGINE Bus STARTING
CherryPy Checker:
The application mounted at '' contains the following config entries, which are only allowed in site-wide config. Move them to a [global] section and pass them to cherrypy.config.update() instead of tree.mount().
[/] server.socket_host = 0.0.0.0
[/] server.socket_port = 6060

[12/Jun/2024:11:18:13] ENGINE Started monitor thread 'Autoreloader'.
INFO:cherrypy.error:[12/Jun/2024:11:18:13] ENGINE Started monitor thread 'Autoreloader'.
[12/Jun/2024:11:18:13] ENGINE Serving on http://0.0.0.0:6060
INFO:

https://78b5-34-168-62-102.ngrok-free.app -> http://127.0.0.1:6060
127.0.0.1 - - [12/Jun/2024:11:18:19] "GET / HTTP/1.1" 200 220 "https://colab.research.google.com/" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36"


INFO:cherrypy.access.133479284082336:127.0.0.1 - - [12/Jun/2024:11:18:19] "GET / HTTP/1.1" 200 220 "https://colab.research.google.com/" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36"


127.0.0.1 - - [12/Jun/2024:11:18:20] "GET /favicon.ico HTTP/1.1" 200 1406 "https://78b5-34-168-62-102.ngrok-free.app/" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36"


INFO:cherrypy.access.133479284082336:127.0.0.1 - - [12/Jun/2024:11:18:20] "GET /favicon.ico HTTP/1.1" 200 1406 "https://78b5-34-168-62-102.ngrok-free.app/" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36"
[12/Jun/2024:11:18:39] HTTP 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/cherrypy/_cprequest.py", line 638, in respond
    self._do_respond(path_info)
  File "/usr/local/lib/python3.10/dist-packages/cherrypy/_cprequest.py", line 697, in _do_respond
    response.body = self.handler()
  File "/usr/local/lib/python3.10/dist-packages/cherrypy/lib/encoding.py", line 223, in __call__
    self.body = self.oldhandler(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/cherrypy/_cpdispatch.py", line 54, in __call__
    return self.callable(*self.args, **self.kwargs)
  File "<ipython-input-23-d3d7e90ca48d>", line 30, in upload_file
    file = request.files['audio_file']
  F

127.0.0.1 - - [12/Jun/2024:11:18:39] "POST /upload_file HTTP/1.1" 500 2037 "https://78b5-34-168-62-102.ngrok-free.app/" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36"


INFO:cherrypy.access.133479284082336:127.0.0.1 - - [12/Jun/2024:11:18:39] "POST /upload_file HTTP/1.1" 500 2037 "https://78b5-34-168-62-102.ngrok-free.app/" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36"
[12/Jun/2024:11:19:06] ENGINE Keyboard Interrupt: shutting down bus
INFO:cherrypy.error:[12/Jun/2024:11:19:06] ENGINE Keyboard Interrupt: shutting down bus
[12/Jun/2024:11:19:06] ENGINE Bus STOPPING
INFO:cherrypy.error:[12/Jun/2024:11:19:06] ENGINE Bus STOPPING
[12/Jun/2024:11:19:07] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('0.0.0.0', 6060)) shut down
INFO:cherrypy.error:[12/Jun/2024:11:19:07] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('0.0.0.0', 6060)) shut down
[12/Jun/2024:11:19:07] ENGINE Stopped thread 'Autoreloader'.
INFO:cherrypy.error:[12/Jun/2024:11:19:07] ENGINE Stopped thread 'Autoreloader'.
[12/Jun/2024:11:19:07] ENGINE Bus STOPPED
INFO:cherrypy.error:[12/Jun/2024:11:19:07